# Conditions d'utilisation 
Pour que **InPoDa** fonctionne correctement vérifiez que les conditions suivantes doivent être réunies :
- Installation des modules : ```Pandas```, ```Textblob```, ```Spacy```, ```Translate```, ```Languedetect```, (```Gradio```?)
- Le fichier ***"aitweets.json"*** doit être dans le même dossier que **InPoDa**

# Supression des caractères spéciaux

In [171]:
# Etat : Fonctionne parfaitement 

def SuprCaracterSpe(Chaine):  # Prend en entrée une chaine de caractères 
    ChaineNettoyee = ''
    indice = 0
    while indice < len(Chaine):
        if (Chaine[indice] == 'R') and (indice < len(Chaine)-2) and (Chaine[indice] + Chaine[indice+1] == 'RT'): # On suprime les "RT"
            indice += 3
            while (indice < len(Chaine)-1) and Chaine[indice] != ' ':
                indice+=1
            ChaineNettoyee += ' '
            indice += 1
        elif (Chaine[indice].isalnum() == True):
            ChaineNettoyee += Chaine[indice]
            indice += 1
        elif (Chaine[indice] == "\ "[:-1]) and (indice < len(Chaine)-2) and (Chaine[indice] + Chaine[indice+1] == '\n'): # On suprime les sauts de ligne
            ChaineNettoyee += ' '
            indice += 1
        elif (Chaine[indice] == '@') and (indice < len(Chaine)-2):  # On suprime les pseudonymes (donc les arobases) 
            indice += 1
            while (indice < len(Chaine)-1) and Chaine[indice] != ' ':
                indice += 1
            ChaineNettoyee += ' '
            indice += 1
        else:
            ChaineNettoyee += ' '
            indice += 1
    return ChaineNettoyee     # Renvoie la chaine de cractères sans les caractères spéciaux

# Création de la classe ***Tweet***

In [172]:
class Tweet:
    
    def __init__(self, chaine='TweetParDéfaut') :
        # 'chaine' doit être un 'str'
        self.chaine = chaine # Reconversion du Tweet en 'str'

        def Hashtags(chaine): # Entrée : Chaîne de caractères
            ListeH = []
            SousChaine = chaine.split()
            for elem in SousChaine:
                if elem.startswith('#') and len(elem)>1:
                    elem2 = ''
                    indice = 1
                    while indice < len(elem):
                        if (elem[indice] == '_') or (elem[indice].isalnum() == True):
                            elem2 += elem[indice]
                            indice += 1
                        else:
                            indice = len(elem)
                    ListeH.append(elem[0]+elem2)
            return ListeH   # Renvoie une liste contenant les # du Tweet
        
        self.Hashtags = Hashtags(chaine)

        def Arobases(chaine): # Prend en entrée un str
            ListeA = []
            SousChaine = chaine.split()
            for elem in SousChaine:
                if elem.startswith('@') and len(elem)>1:
                    elem2 = ''
                    indice = 1
                    while indice < len(elem):
                        if (elem[indice] == '_') or (elem[indice].isalnum() == True):
                            elem2 += elem[indice]
                            indice += 1
                        else:
                            indice = len(elem)
                    ListeA.append(elem[0]+elem2)
            return ListeA   # Renvoie une liste contenant les @ du Tweet 

        self.Arobases = Arobases(chaine)

        def Sentiment(Tweet): # Prend en entrée une chaîne de caractères
            from textblob import TextBlob
            blob = TextBlob(Tweet)
            polarite = blob.sentiment.polarity
            if polarite > 0:
                return 'Positif'
            elif polarite < 0:
                return 'Negatif'
            else:
                return 'Neutre' # Sortie : Chaîne de caractère ('str')
            
        self.Sentiment = Sentiment(chaine)
    
    def __repr__(self):
        return self.chaine
        

chaine = "Bon @jour Je suis la"
Tweet1 = Tweet(chaine)
type({})

dict

# Création de la classe ***Inpoda***

In [173]:
from pandas import DataFrame

class Inpoda:
    def __init__(self, Dico=None, DF=None):
        if DF is None:
            DF = DataFrame([{"DataFrameParDéfaut": True}])
        elif Dico is None:
            Dico = {"DicoParDéfaut": True}
        self.Dico = Dico
        assert type(self.Dico) == type({})
        self.DF = DF
        assert type(self.DF) == type(DataFrame())


    def TopKH(self, K=5, DicoH=None):# Entrée : {K : 'int' (si K>0 on aura un tri décroissant, si K<0 on aura un tri croissant)
                                            #    DicoH : 'dict' (Dictionnaire contenant tous les Hashtags de la base de données et leur nombre)}
        from pandas import DataFrame
        if DicoH is None:
            DicoH = self.Dico
        DFKH = DataFrame(DicoH.items(), columns=['#', 'Nombre de Tweets'])
        if K > 0:
            DFKH = DFKH.sort_values(by='Nombre de Tweets', ascending=False) # On tri le Dataframe selon les valeurs de la deuxième colonne 'RetweetCount' et on precise 'ascending=False' pour l'ordre décroissant
        else:
            DFKH = DFKH.sort_values(by='Nombre de Tweets', ascending=True) # Inversement on tri le DataFrame par ordre Décroissant
        return DFKH.head(abs(K))# On passe la valeur absolue de K dans la méthode .head() pour s'assurer qu'elle affiche correctement le Top demandé
                # Sortie : 'DataFrame' (de deux colonnes contenant les Utilisateurs mentionnés et le Nombre de fois qu'ils ont été mentionné)


    def TopKU(self, K=5, DFP=None):   # Entrée : {K : 'int' (si K>0 on aura un tri décroissant, si K<0 on aura un tri croissant)
                                     #          DFP : 'DataFrame' (Le DataFrame principal)}
        from pandas import concat
        if DFP is None:
            DFP = self.DF
        Taille = DFP["RetweetCount"].shape
        for i in range(Taille[0]):
            valeur = int(DFP["RetweetCount"].loc[i])  # On converti les 'str' de la colonne "RetweetCount" en entier pour correctement les trier
            DFP["RetweetCount"].loc[i] = valeur       
        DFKU = concat([DFP["Utilisateurs"], DFP["RetweetCount"]], axis=1)
        if K > 0:
            DFKU = DFKU.sort_values(by="RetweetCount", ascending=False) # On tri le Dataframe selon les valeurs de la deuxième colonne 'RetweetCount' et on precise 'ascending=False' pour l'ordre décroissant
        else:
            DFKU = DFKU.sort_values(by="RetweetCount", ascending=True) # Inversement on tri le DataFrame par ordre Décroissant
        return DFKU.head(abs(K))  # On passe la valeur absolue de K dans la méthode .head() pour s'assurer qu'elle affiche correctement le Top demandé
         # Sortie : 'DataFrame' (de deux colonnes contenant les Utilisateurs et le Nombre de fois qu'ils ont tweeté) 


    def TopKA(self, K=5, DicoA=None):  # Entrée : {K :'int' (si K>0 on aura un tri décroissant, si K<0 on aura un tri croissant)
                        #          DicoA : 'dict' (Dictionnaire contenant tous les Arobases de la base de données et leur nombre)}
        from pandas import DataFrame
        if DicoA is None:
            DicoA = self.Dico
        DFKA = DataFrame(DicoA.items(), columns=['@', 'Nombre de Mentions'])
        if K > 0:
            DFKA = DFKA.sort_values(by='Nombre de Mentions', ascending=False) # On tri le Dataframe selon les valeurs de la deuxième colonne 'Nombre' et on precise 'ascending=False' pour l'ordre décroissant
        else:
            DFKA = DFKA.sort_values(by='Nombre de Mentions', ascending=True) # Inversement on tri le DataFrame par ordre Décroissant  
        return DFKA.head(abs(K))  # On passe la valeur absolue de K dans la méthode .head() pour s'assurer qu'elle affiche correctement le Top demandé
            # Sortie : 'DataFrame' (de deux colonnes contenant les Utilisateurs et le Nombre de fois qu'ils ont été mentionnés) 

    def NbPostUtilisateur(self, Utilisateur="Lugol", DFP=None) : # Entrée : {Utilisateur : 'str' (Le Nom de l'utilisateur)
                                                                  #          DFP : 'DataFrame' (DataFrame principal)}
        if DFP is None:
            DFP = self.DF
        Taille = DFP.shape
        for i in range(Taille[0]):
            if DFP["Utilisateurs"].loc[i] == Utilisateur:
                return (Utilisateur, DFP["RetweetCount"].loc[i])  
        return (Utilisateur, 0)   
            # Sortie : 'tuple' (Utilisateur, nombre de Publications)


    def NbPostHashtags(self, hashtag='#ParDéfaut', DicoH=None): # Entrée : { hashtag : 'str' (Le Hashtag recherché)
                                                                #            DicoH : 'dict' (Le dictionnaire des Hastags)}
        if DicoH is None:
            DicoH = self.Dico
        if hashtag in DicoH:
            return (hashtag, DicoH[hashtag])
        else:
            return (hashtag, 0) # Sortie : 'tuple' (#, nombre de publications dans lequel il est mentionné)
    

    def TweetsUtilisateur(self, Utilisateur="Lugol", DFP=None): # Entrée : { Utilisateur : 'str' (Nom de l'Utilisateur)
                                                                #            DFP : 'DataFrame' (DataFrame principal) }
        from pandas import DataFrame
        if DFP is None:
            DFP = self.DF
        if Utilisateur[0] != "@":
            Utilisateur = "@" + Utilisateur
        ExpressionR = 'RT ' + Utilisateur
        ListeTweets = []
        Taille = DFP["TweetText"].shape
        for indice in range(Taille[0]):
            tweet = DFP["TweetText"].loc[indice]
            tweet = tweet.chaine
            if (ExpressionR in tweet) and (tweet[len(ExpressionR)+2:] not in ListeTweets):
                ListeTweets.append(tweet[len(ExpressionR)+2:])
            elif (DFP["Utilisateurs"].loc[indice] == Utilisateur):
                ListeTweets.append(tweet)
        if ListeTweets == []:
            return DataFrame([{"Tweets postés par : " + Utilisateur:"Erreur : Aucun Tweet posté"}])
        else:
            return DataFrame({"Tweets postés par : " + Utilisateur:ListeTweets}) 
                # Sortie : 'DataFrame' contenant les à chaque ligne un tweet de l'utilisateur (Tweet brut) 
                #          (attention l'index des Tweets de DFTU ne correspondent pas à leur index dans DFP)


    def TweetsMentionUtilisateur(self, Utilisateur = "Lugol", DFP = None): # Entrée : { Utilisateur : 'str' (Nom d'Utilisateur)
                                                                           #            DFP : 'DataFrame' (DataFrame principal)
        from pandas import DataFrame
        if DFP is None:
            DFP = self.DF
        Taille = DFP.shape
        ListeIndiceTweets = []
        for i in range(Taille[0]):
            ListeA = DFP["@"].loc[i]
            if Utilisateur in ListeA:
                ListeIndiceTweets.append(i)
        DFTMU = DataFrame(DFP["TweetText"].iloc[ListeIndiceTweets])
        DFTMU = DFTMU.rename(columns={"TweetText":"Tweets mentionnant : " + Utilisateur})  # On renomme la colonne pour plus de clarté 
        return DFTMU     # Sortie : 'DataFrame' contenant à chaque ligne les Tweets mentionnant l'Utilisateur, 
                                # l'index de chaque ligne correpond à celui du Tweet brut stocké dans DFP 


    def UtilisateurMentionHashtag(self, Hashtag = "#ParDéfaut", DFP = None): # Entrée : { Hastag : 'str' (Hashtag)
                                             #            DFP : 'DataFrame' (DataFrame principal) }
        from pandas import DataFrame
        if DFP is None:
            DFP = self.DF
        Taille = DFP.shape
        ListeIndiceTweets = []
        for i in range(Taille[0]):
            ListeH = DFP["#"].loc[i]
            if Hashtag in ListeH:
                ListeIndiceTweets.append(i)
        if ListeIndiceTweets == []:
            return DataFrame([{"Utilisateurs":"Erreur : Votre Hashtag n'existe pas"}])
        else:
            return DataFrame(DFP["Utilisateurs"].iloc[ListeIndiceTweets]) 
              # Sortie : 'DataFrame' contenant à chaque ligne un Utilisateur qui à mentionner le Hashtag dans son tweet, 
              #          l'index correspond à celui de l'Utilisateur dans DFP


    def UtilisateurMentionUtilsateur(self, Utilisateur = "Lugol", DFP = None):  # Entrée : { Utilisateur : 'str' (Nom d'Utilisateur)
                                                     #            DFP : 'DataFrame' (DataFrame principal)
        from pandas import DataFrame
        if DFP is None:
            DFP = self.DF
        Taille = DFP.shape
        for i in range(Taille[0]):
            if DFP["Utilisateurs"].loc[i] == Utilisateur:
                return DataFrame({Utilisateur + " a mentionné :":DFP["@"].loc[i]})   
        return DataFrame([{"Utilisateurs":"Erreur : Votre Utilisateur n'existe pas"}]) 
            # Sortie : 'DataFrame'


    def __repr__(self):
        if (self.Dico != {"DicoParDéfaut": True}) and (self.Dico != {}):
            return repr(self.Dico)
        else:
            return repr(self.DF)


# Récupération des Tweets du fichier Json → Json (Zone d'aterrissage)

In [174]:
# Etat : Fonctionne parfaitement

def ZoneAterrisage(NomFichier):
    from json import loads
    with open(NomFichier, 'r') as fs, open("Zone d’atterrissage.json", 'w') as fd:
        for ligne in fs:
            dicoL = loads(ligne)
            tweet = dicoL["TweetText"]
            tweetN = SuprCaracterSpe(tweet)  # Suprime tous les caractères spéciaux, les RT et les pseudonymes
            fd.write(f'{tweetN}\n')     # On écrit le Tweet nettoyé dans "Zone d’atterrissage.json"

fichier = "aitweets.json"
ZoneAterrisage(fichier)

# Récupération des Données du fichier Json → DataFrame ('DFP')

In [175]:
# Etat : Fonctionne parfaitement

def RecupDonneesP(NomFichier):  # Prend en entier le nom du fichier à analyser
    from pandas import DataFrame
    from pandas import concat
    from json import loads
    with open(NomFichier, 'r') as fs:
        ligne = fs.readline()
        dicoL = loads(ligne)
        tweet = Tweet(dicoL["TweetText"])
        dicoL["TweetText"] = tweet
        dicoL["@"] = tweet.Arobases
        dicoL["#"] = tweet.Hashtags
        dicoL["Sentiment"] = tweet.Sentiment
        DFF = DataFrame([dicoL])   # On initialise le DatFrame qui va stocker en local les données de la base
        for ligne in fs:
            dicoL = loads(ligne)
            tweet = Tweet(dicoL["TweetText"])
            dicoL["TweetText"] = tweet
            dicoL["@"] = tweet.Arobases
            dicoL["#"] = tweet.Hashtags
            dicoL["Sentiment"] = tweet.Sentiment
            DF2 = DataFrame([dicoL])
            DFF = concat([DFF,DF2])  
    DFF.index = list(i for i in range(0,1708))               
    return DFF

fichier = "aitweets.json"
DFP = RecupDonneesP(fichier)   # DFP correspond au DataFrame principal, dont les colonnes sont 'id','TweetText', etc
DFP

,id,AuthorLocation,CreatedAt,RetweetCount,TweetLanguage,TweetText,@,#,Sentiment
0,1415291904850153474,,2021-07-14T12:47:39Z,0,en,"@____bruvteresa_ According to research, NASA c...",[@____bruvteresa_],[],Positif
1,1415291947560828933,Mysore and BERLIN,2021-07-14T12:47:49Z,2,en,RT @HDataSystems: Artificial Intelligence and ...,[@HDataSystems],"[#hdatasystems, #Artificia]",Negatif
2,1415291877897605120,,2021-07-14T12:47:33Z,246,en,RT @adgpi: Army Technology Board conducted the...,[@adgpi],[],Negatif
3,1415291886860967940,,2021-07-14T12:47:35Z,1,en,"RT @pacorjo: According to a recent survey, the...",[@pacorjo],[],Positif
4,1415291968700264450,Internet,2021-07-14T12:47:54Z,20,en,RT @HarbRimah: Making AI Sing https://t.co/FJo...,[@HarbRimah],"[#MachineLearning, #DataScience, #Python, #AI,...",Neutre
...,...,...,...,...,...,...,...,...,...
1703,1421408743770791938,"Liverpool, England",2021-07-31T09:53:47Z,1,en,RT @innomaticshyd: Top 10 Real world Artificia...,[@innomaticshyd],[],Positif
1704,1421424066305605634,,2021-07-31T10:54:40Z,0,en,Iowa State part of U.S. National Science Found...,[],"[#ArtificialIntelligence, #IIoT, #GenerativeAd...",Positif
1705,1421423882427371521,127.0.0.1,2021-07-31T10:53:57Z,17,en,RT @intellimetri: Human Assisted #ArtificialIn...,"[@intellimetri, @nerdgirlz, @forbes, @ForbesBR...","[#ArtificialIntelligence, #AI]",Neutre
1706,1421423971858149377,"Singapore, Singapore",2021-07-31T10:54:18Z,12,en,RT @IainLJBrown: Artificial Intelligence learn...,[@IainLJBrown],[#Artificia],Positif


# Dictionnaire des Hashtags ('#')

In [176]:
def DicoHashtags(DF):   # Entrée : DataFrame (la colonne des tweets)
    DicoH = {}
    from pandas import DataFrame
    Taille = DF.shape  # Tuple contenant (le nombre de ligne, le nombre de colonnes)
    for i in range(Taille[0]):
        tweet = DF.iloc[i]
        ListeHashtags = tweet.Hashtags 
        for h in ListeHashtags:
            if h in DicoH:
                nombre = DicoH[h]
                nombre +=1
                DicoH[h] = nombre
            else:
                DicoH[h] = 1
    del DicoH['#']    # Il y a 33 # incomplets dans la base de données, ils sont pris en compte par la fonction, on prend donc le soin de suprimer cette clé du dico           
    return DicoH      # Renvoie un Dictionnaire contenant tous les Hashtags en clé et le nombre de fois qu'ils sont tweeté en valeur

DicoH = DicoHashtags(DFP["TweetText"])

# Dictionnaire des Arobases @

In [177]:
def DicoArobases(DF):   # Entrée : DataFrame (la colonne des tweets)
    DicoA = {}
    from pandas import DataFrame
    Taille = DF.shape  # Tuple contenant (le nombre de ligne, le nombre de colonnes)
    for i in range(Taille[0]):
        tweet = DF.iloc[i]
        ListeArobases = tweet.Arobases
        for h in ListeArobases:
            if h in DicoA:
                nombre = DicoA[h]
                nombre +=1
                DicoA[h] = nombre
            else:
                DicoA[h] = 1 
    del DicoA['@']               
    return DicoA      # Renvoie un Dictionnaire contenant tous les Hashtags en clé et le nombre de fois qu'ils sont tweeté en valeur

DicoA = DicoArobases(DFP["TweetText"])

# Utilisateurs

L'enjeux est ici de compléter notre base de données incomplète. En effet notre base de données comporte pour chaque tweet un champ **ID** mais aucun "Pseudo" ou "Nom d'Utilisateur".
C'est pourquoi nous allons ajouter à notre **DataFrame** principal ce nouveau champ "Utilisateurs".

La création d'Utilisateurs est essentielle au bon déroulement de certaines opérations d'Analyses, notamment pour 'Les utilisateurs mentionnés par un utilisateur spécifique'. Il est donc impératif que tous les utilisateurs mentionnés dans les tweets soient présent au sein de la base de données, donc dans le champ "**Utilisateurs**" du DataFrame **DFP**.

In [178]:
# Etat : Fonctionne parfaitement

def CreationUtilisateurs(ListeD, nb, DFP):  # Entrée : Liste Utilisateurs mentionnés ('dict_keys'), nombre ('int'), DataFrame principal (DFP)
    from random import randint
    from pandas import DataFrame
    from pandas import concat
    ListeUtilisateurs = list(ListeD)
    ListeUtilisateurs.reverse()    # On inverse la liste des utilisateurs mentionnés pour éviter qu'un utilisateur se mentitonne lui même
    for i in range(nb):
        Lettre1 = chr(randint(65, 90))   # Intervalle Unicode des lettres Majuscules de l'alphabet latin
        Lettre2 = chr(randint(97, 122))  
        Lettre3 = chr(randint(97, 122))  # Intervalle Unicode des lettres Minuscules de l'alphabet latin
        Lettre4 = chr(randint(97, 122))
        Chiffre1 = chr(randint(48, 57))  # Intervalle Unicode des chiffres 
        Utilisateur = '@' + Lettre1 + Lettre2 + Lettre3 + Lettre4 + Chiffre1  # On ajoute '@' en début de Nom d'utilisateur pour s'assurer de l'homogéité des données.
        ListeUtilisateurs.append(Utilisateur)
    DFUtilisateurs = DataFrame({"Utilisateurs": ListeUtilisateurs})
    DFP = concat([DFUtilisateurs, DFP], axis=1)
    return DFP
   
DFP = CreationUtilisateurs(DicoA.keys(), (1708-len(DicoA)), DFP)

# Utilisation de la classe ***Inpoda***
Maintenant que les trois variables ***DicoA***, ***DicoH*** et ***DFP*** ont été complétées on peut les transformer en objet 'Inpoda' pour réaliser les opérations d'analyses.

In [179]:
DFP = Inpoda(DF= DFP)
DicoH = Inpoda(Dico= DicoH)
DicoA = Inpoda(Dico= DicoA)

# Fonctionnalitées Suplémentaires :

- Actualisation manuelle des données 

In [180]:
# Etat : Fonctionne parfaitement

def Refresh():
    fichier = "aitweets.json"
    ZoneAterrisage(fichier)

Refresh()

# Les TOP-K 

- Top K hashtags

In [181]:
DicoH.TopKH(5)

,#,Nombre de Tweets
11,#ArtificialIntelligence,207
5,#AI,204
2,#MachineLearning,86
27,#ai,52
3,#DataScience,50


- Top K Utilisateurs

In [182]:
DFP.TopKU(5)

,Utilisateurs,RetweetCount
1194,@Rjbm5,10327
1585,@Qrno7,6481
1587,@Slsr5,6481
735,@LuxuryOpinions,1660
1047,@SecDef,1060


- Top K utilisateurs mentionnés

In [183]:
DicoA.TopKA(5)

,@,Nombre de Mentions
26,@IainLJBrown,112
55,@Paula_Piccard,49
18,@nigewillson,24
679,@JoshuaBarbeau,17
977,@sankrant,17


# Le Nombre de publication par Utilisateur/Hashtag

- Le nombre de publications par utilisateur

In [184]:
DFP.NbPostUtilisateur(Utilisateur="@adgpi")

('@adgpi', 27)

- Le nombre de publications par hashtag

In [185]:
DicoH.NbPostHashtags(hashtag= '#AI')

('#AI', 204)

# Les dernières analyses demandées

- L’ensemble de tweets d’un utilisateur spécifique

In [186]:
DFP.TweetsUtilisateur(Utilisateur="@Gizchina")

,Tweets postés par : @Gizchina
0,OPPO releases its first 6G white paper. A syst...
1,"RT @sfchronicle: “Intellectually, I know it’s ..."


- L’ensemble de tweets mentionnant un utilisateur spécifique

In [187]:
DFP.TweetsMentionUtilisateur(Utilisateur="@intratio")

,Tweets mentionnant : @intratio
75,RT @intratio: $MDVL https://t.co/rso4WTkGcW Me...
400,RT @intratio: https://t.co/GGI5g1FRD3 Kinnate ...
843,RT @intratio: $CUE https://t.co/N3KHKWvbdp Cue...


- Les utilisateurs mentionnant un hashtag spécifique

In [188]:
DFP.UtilisateurMentionHashtag(Hashtag='#AI')

,Utilisateurs
4,@nerdgirlz
8,@arXiv_Daily
10,@DecisionsSmart
42,@frontiersin
58,@TACHOUHONER
...,...
1679,@Opsy7
1681,@Fbmj2
1684,@Xhos8
1699,@Sqst6


- Les utilisateurs mentionnés par un utilisateur spécifique

In [189]:
DFP.UtilisateurMentionUtilsateur(Utilisateur="@xoce_q")

,@xoce_q a mentionné :
0,@codewithibrahim


# Interface Graphique de Inpoda

- Gradio

# Ressources suplémentaires
Des ressources suplémentaire sont disponible sur le Github dont le lien est ci dessous. Vous y trouverez notamment un diagramme détaillant le fonctionnement de **InPoDa** ainsi que le détail de la répartition des taches. 

Github : https://github.com/LugolBis/Projet-IN304 
